In [1]:
import pandas as pd
import numpy as np
from codebase.file_utils import (
    save_obj,
    load_obj,
)
from codebase.plot import plot_density, plot_line, get_post_df
import altair as alt
from codebase.classes_data import Data
from codebase.ibis import exp_and_normalise
from run_smc2 import run_smc2
from codebase.file_utils import (
    save_obj,
    load_obj,
    make_folder,
    path_backslash
)
from codebase.ibis import post_process_sign
from pdb import set_trace


alt.data_transformers.disable_max_rows()


DataTransformerRegistry.enable('default')

In [12]:
log_dir = 'log/20210204_224313_run50//'
exp_data = load_obj('data', log_dir)
t = load_obj('t', log_dir)
print(t)

199


##

In [13]:
particles = load_obj('particles', log_dir)
# particles.particles[5].particles['beta']

In [14]:
particles.resample_particles_bundles()

particles.gather_latent_variables_up_to_t(
    t+1, 
    exp_data.get_stan_data_upto_t(t+1)
)

particles.jitter_bundles_and_pick_one(exp_data.get_stan_data_upto_t(t + 1))
particles.check_latent_particles_are_distinct()

particles.jitter(exp_data.get_stan_data_upto_t(t + 1))

In [22]:
ps_smc2 = dict()
for name in particles.param_names:
    ps_smc2[name] = particles.extract_particles_in_numpy_array(name)


In [27]:
ps_smc2 = post_process_sign(ps_smc2)


## Plot

In [28]:
param = 'beta'
df = get_post_df(ps_smc2[param]) 
df['source'] = 'smc2'
plot_density(df, height=100)

alt.Chart(...)

## Plot MCMC samples

In [29]:
param = 'beta'
df = get_post_df(ps_smc2[param])
df_quant = df.groupby(['row', 'col'])[['value']].quantile(0.025).reset_index()
df_quant.rename({'value':'q1'}, axis=1, inplace=True)
df_quant2 = df.groupby(['row', 'col'])[['value']].quantile(0.975).reset_index()
df_quant2.rename({'value':'q2'}, axis=1, inplace=True)

df = df_quant.merge(df_quant2, on=['row', 'col'])

# simple quantile chart
df['source'] = 'smc2'
c1 = alt.Chart(df).mark_bar(opacity=0.6).encode(
        alt.X('q1', title=None),
        alt.X2('q2', title=None),
        alt.Row('row'),
        alt.Column('col'),
        alt.Color('source')
)
c1

alt.Chart(...)

In [30]:
df['index'] = 'r_' + df.row.astype(str)+'.c_'+df.col.astype(str)
df = df.loc[:,['index', 'q1', 'q2']]
df['source'] = 'smc2'
dd = pd.DataFrame(exp_data.raw_data['beta'], columns=['data'])
dd['col'] = 0
dd['row'] = np.arange(6)
dd['index'] = 'r_' + dd.row.astype(str)+'.c_'+dd.col.astype(str)
dd = dd.loc[:,['index', 'data']]
plot_data = df.merge(dd, on=['index'])

In [31]:
c1 = alt.Chart(plot_data).mark_bar(opacity=0.6).encode(
    alt.X('q1', title=None,  scale=alt.Scale(domain=[-2,2])),
    alt.X2('q2', title=None),
    alt.Color('source'),    
)
    

c2 = alt.Chart(plot_data).mark_point(opacity=1, color='red').encode(
        alt.X('data', title=None),
)
(c1+c2).facet(
       'index',
    columns=1
    )


alt.FacetChart(...)

In [32]:
load_obj('marg_lklhd', log_dir)

8.840333720190472